<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-08 09:30:05
-------------------
qualified stocks: 94
with latest results: 87
still star stocks: 47


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  6.31 L
Current:  1.24 C
-------------------
Today PnL: 34.27 K (0.28%)
Current PnL: -21.82 L (-15.47%)
CY Booked + Current PnL: -18.99 L (-13.46%)
-------------------
Total profit:  2.92 L
Total loss:  -24.74 L
-------------------
Total Booked + Current PnL: 7.99 L (7.01%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.29%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.05 C
Est FTT PnL: 81.02 L (65.5%)
-------------------
Est LTT:  2.66 C
Est LTT PnL: 1.42 C (114.92%)
Deployed:  1.14 C
Current:  1.24 C
CAGR/XIRR %: 5.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.14,23.79,X-MC,98.94,142335.0,27557.0,8170.0,1.36,24.01,5.74,31.13,203.0,3.37,1.21,38.65,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-2.11,H-LC,88.30,145362.0,19657.0,16339.0,-0.80,15.64,11.24,28.63,116.0,1.20,1.24,29.50,AR,ATH,INSURANCE
57,PIDILITIND,2504.06,3576.00,-14.69,X-LC,32.98,89007.0,13885.0,18273.0,-0.28,18.48,20.53,42.81,66.0,0.76,0.76,11.63,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-17.60,M-LC,35.11,206501.0,5022.0,22199.0,0.41,2.49,10.75,13.51,106.0,0.23,1.76,13.13,XY25,NTT,BANKS
35,ICICIGI,1852.76,2268.71,-18.26,H-LC,40.43,98420.0,-1629.0,24093.0,-0.19,-1.63,24.48,22.45,93.0,-0.07,0.84,19.41,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,798.82,1065.00,-53.87,M-LC,92.55,143705.0,-20053.0,74626.0,3.04,-12.25,51.93,33.32,1.0,-0.27,1.22,20.91,XY24,NTT,AUTO
9,ATULAUTO,624.44,844.00,3429.73,M-SC,7.45,109988.0,-37380.0,89200.0,2.45,-25.36,81.10,35.16,188.0,-0.42,0.94,12.34,XY24,NTT,AUTO
19,CERA,9059.90,10420.45,79.83,X-SC,29.79,55285.0,-35314.0,48922.0,2.35,-38.98,88.49,15.02,50.0,-0.72,0.47,8.78,X40N,ATH,CERAMICS
54,NHPC,82.08,115.00,195.69,M-MC,23.40,143298.0,2285.0,54267.0,2.08,1.62,37.87,40.11,211.0,0.04,1.22,17.15,XY24,NTT,POWER
79,TANLA,991.56,1963.11,-57.60,H-SC,71.28,107649.0,-111486.0,326198.0,2.06,-50.88,303.02,97.98,2.0,-0.34,0.92,16.60,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SATIN,199.77,274.00,-19.84,H-SC,82.98,159020.0,-36954.0,109772.0,-3.42,-18.86,69.03,37.16,118.0,-0.34,1.35,18.75,XY24,NTT,FINANCE
20,COFFEEDAY,59.14,80.00,-59.76,L-SC,80.85,59021.0,-54528.0,94581.0,-2.01,-48.02,160.25,35.27,151.0,-0.58,0.50,44.45,XR,NTT,HOTELS
22,DABUR,505.20,735.00,-15.99,X-MC,44.68,185074.0,-11954.0,101569.0,-1.57,-6.07,54.88,45.49,81.0,-0.12,1.58,5.13,XY24,BTT,FMCG
21,COLPAL,2837.00,3808.06,6.04,X-MC,62.77,129810.0,-12040.0,60595.0,-1.47,-8.49,46.68,34.23,36.0,-0.20,1.11,10.45,X40,ATH,FMCG
65,ROUTE,1558.40,2249.26,-10.05,M-SC,41.49,62588.0,-40266.0,85864.0,-1.17,-39.15,137.19,44.33,19.0,-0.47,0.53,2.94,SR,ATH,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,798.82,1065.00,-53.87,M-LC,92.55,143705.0,-20053.0,74626.0,3.04,-12.25,51.93,33.32,1.0,-0.27,1.22,20.91,XY24,NTT,AUTO
79,TANLA,991.56,1963.11,-57.60,H-SC,71.28,107649.0,-111486.0,326198.0,2.06,-50.88,303.02,97.98,2.0,-0.34,0.92,16.60,AR,ATH,IT
2,ABB,5319.37,8953.12,-40.84,H-LC,64.89,134212.0,1228.0,89613.0,-0.54,0.92,66.77,68.31,3.0,0.01,1.14,9.44,AR,ATH,ELECTRICAL
24,DLF,684.82,1031.70,-26.21,H-LC,76.60,102038.0,-685.0,52713.0,0.15,-0.67,51.66,50.65,4.0,-0.01,0.87,11.87,AR,ATH,REALTY
10,AWL,361.39,485.00,-69.11,H-MC,9.57,135857.0,-50259.0,113916.0,0.82,-27.00,83.85,34.20,6.0,-0.44,1.16,11.90,XY24,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.14,23.79,X-MC,98.94,142335.0,27557.0,8170.0,1.36,24.01,5.74,31.13,203.0,3.37,1.21,38.65,AR,ATH,TYRES
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,93.62,184964.0,33758.0,31906.0,0.89,22.33,17.25,43.43,45.0,1.06,1.58,28.21,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NHPC,82.08,115.00,195.69,M-MC,23.40,143298.0,2285.0,54267.0,2.08,1.62,37.87,40.11,211.0,0.04,1.22,17.15,XY24,NTT,POWER
58,POLYCAB,5005.92,7541.35,16.31,H-MC,89.36,164976.0,24810.0,46177.0,0.09,17.70,27.99,50.65,74.0,0.54,1.41,25.91,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,93.62,184964.0,33758.0,31906.0,0.89,22.33,17.25,43.43,45.0,1.06,1.58,28.21,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.00,-4.86,H-SC,65.96,155246.0,3882.0,41901.0,0.53,2.56,26.99,30.25,108.0,0.09,1.32,22.87,X40N,NTT,FINANCE
18,CAMS,3643.00,5250.99,-6.33,H-SC,47.87,103734.0,1730.0,43288.0,1.70,1.70,41.73,44.14,26.0,0.04,0.88,21.48,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-6.33,H-SC,47.87,103734.0,1730.0,43288.0,1.70,1.70,41.73,44.14,26.0,0.04,0.88,21.48,XR,ATH,MISC
93,WIPRO,243.46,420.00,-11.98,M-LC,17.02,151175.0,230.0,109224.0,-0.09,0.15,72.25,72.51,77.0,0.00,1.29,13.82,XR,NTT,IT
91,VOLTAS,1272.00,1929.20,-18.85,H-MC,6.38,138094.0,-1826.0,74115.0,0.92,-1.31,53.67,51.67,9.0,-0.02,1.18,4.30,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-28.40,M-MC,24.47,124861.0,-4291.0,91361.0,0.49,-3.32,73.17,67.42,99.0,-0.05,1.06,21.44,XR,NTT,BANKS
1,AARTIIND,487.04,919.00,21.89,M-SC,100.00,77020.0,-4316.0,76450.0,0.60,-5.31,99.26,88.69,114.0,-0.06,0.66,29.88,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,-42.86,M-SC,75.53,110646.0,-31833.0,51351.0,0.52,-22.34,46.41,13.70,139.0,-0.62,0.94,30.59,X40N,NTT,BANKS
52,MRF,114777.60,150504.14,23.79,X-MC,98.94,142335.0,27557.0,8170.0,1.36,24.01,5.74,31.13,203.0,3.37,1.21,38.65,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-2.11,H-LC,88.30,145362.0,19657.0,16339.0,-0.80,15.64,11.24,28.63,116.0,1.20,1.24,29.50,AR,ATH,INSURANCE
51,MEDANTA,1022.60,1486.00,3.99,X-MC,11.70,183388.0,27953.0,42491.0,-0.36,17.98,23.17,45.32,189.0,0.66,1.56,24.78,XY24,NTT,HEALTHCARE
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,93.62,184964.0,33758.0,31906.0,0.89,22.33,17.25,43.43,45.0,1.06,1.58,28.21,XY24,NTT,MISC


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,21.89,M-SC,100.00,77020.0,-4316.0,76450.0,0.60,-5.31,99.26,88.69,114.0,-0.06,0.66,29.88,XR,NTT,CHEMICALS
52,MRF,114777.60,150504.14,23.79,X-MC,98.94,142335.0,27557.0,8170.0,1.36,24.01,5.74,31.13,203.0,3.37,1.21,38.65,AR,ATH,TYRES
76,SONACOMS,578.05,1006.00,-31.21,M-MC,97.87,89302.0,-11857.0,86748.0,-0.17,-11.72,97.14,74.03,7.0,-0.14,0.76,25.50,AR,BTT,AUTO
90,VIPIND,488.80,718.00,-768.55,H-SC,96.81,63687.0,-31629.0,76323.0,0.96,-33.18,119.84,46.89,39.0,-0.41,0.54,25.62,X40N,NTT,MISC
38,INDUSINDBK,1354.50,1800.00,-35.58,H-MC,95.74,47743.0,-29464.0,54857.0,0.46,-38.16,114.90,32.89,46.0,-0.54,0.41,31.48,XR,NTT,BANKS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,41.72
2,50,69.69


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.93
X40N    14.05
X40     13.67
AR      12.10
XY25    11.38
XR      10.53
X200     1.17
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.43
X-LC    18.75
M-SC    16.37
M-LC     8.16
M-MC     7.68
X-MC     7.05
H-MC     6.46
H-LC     5.60
X-SC     2.62
L-SC     2.28
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,11.32,-13.34,68.93
FMCG,11.13,-3.32,34.95
IT,9.09,-27.76,94.63
FINANCE,7.44,-26.58,83.70
MISC,6.50,-34.94,83.76
PAINTS,5.83,-20.33,55.94
INSURANCE,5.53,-7.96,40.54
ELECTRICAL,4.23,-11.99,69.05
REFINERIES,3.77,7.91,9.06


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3208980.0,28
XR,1307531.0,15
X40N,1251126.0,17
AR,1076375.0,13
X40,768706.0,12
SR,208470.0,2
XY25,200040.0,6
X200,80502.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2335120.0,21
M-SC,1999471.0,20
X-LC,782329.0,12
M-MC,687691.0,7
X-SC,369071.0,4
H-MC,365218.0,6
M-LC,359001.0,6
L-SC,329879.0,4
X-MC,327551.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      968778.0      6
M-SC       XY24      812179.0      6
H-SC       X40N      579728.0      7
M-MC       XY24      484273.0      4
M-SC       XR        447754.0      5
X-LC       X40       396216.0      5
M-SC       X40N      394602.0      5
H-SC       AR        389313.0      2
           XR        307740.0      4
M-LC       XY24      227578.0      4
X-LC       AR        196397.0      3
L-SC       XR        166023.0      2
M-SC       X40       164922.0      2
H-LC       AR        158665.0      3
H-MC       XY24      156750.0      2
X-MC       XY24      144060.0      2
X-SC       XR        130572.0      1
           SR        122606.0      1
           X40N      115893.0      2
X-MC       X40N      114726.0      2
M-LC       XR        109224.0      1
X-LC       XY25      109214.0      3
L-SC       XY24       95039.0      1
M-SC       AR         94150.0      1
M-MC       XR         91361.0      1
H-SC       X40        89561.0      2
M-MC       AR         86748.0      1
M-SC       SR         85864.0      1
X-LC       X200       80502.0      1
H-MC       AR         74115.0      1
L-SC       AR         68817.0      1
X-MC       X40        60595.0      1
H-MC       XR         54857.0      1
           X40N       46177.0      1
H-LC       XY25       43318.0      1
H-MC       X40        33319.0      1
M-MC       XY25       25309.0      1
H-LC       X40        24093.0      1
M-LC       XY25       22199.0      1
X-MC       AR          8170.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 47.0 seconds
